Import and initialize the Earth Engine Python API.

In [54]:
import ee
ee.Initialize()

In [55]:
import ipyleaflet

Import a customized version of IPyLeaflet. The customized version is defined in the file ipyleafletee.py, which is stored in the same folder as this notebook. 

In [56]:
import ipyleafletee

Display the default map. The basemap is based on NOAA's [ETOPO1](https://explorer.earthengine.google.com/#detail/NOAA%2FNGDC%2FETOPO1) elevation dataset.

In [57]:
map1 = ipyleafletee.Map(zoom=2)
map1

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

Examine the map object, to see what it can do.

In [58]:
# Generate a list of methods and properties, suppressing private methods that start with and underscore.
method_and_property_list = ', '.join([prop for prop in dir(map1) if not prop.startswith('_')])
print(method_and_property_list)

add_class, add_control, add_layer, add_traits, attribution_control, basemap, bounce_at_zoom_limits, bounds, bounds_polygon, box_zoom, center, class_own_trait_events, class_own_traits, class_trait_names, class_traits, clear_controls, clear_layers, close, close_all, close_popup_on_click, comm, controls, cross_validation_lock, default_tiles, double_click_zoom, dragging, east, get_manager_state, get_state, get_view_spec, handle_comm_opened, has_trait, hold_sync, hold_trait_notifications, inertia, inertia_deceleration, inertia_max_speed, interact, interpolation, keyboard, keyboard_pan_offset, keyboard_zoom_offset, keys, layers, layout, log, max_zoom, min_zoom, model_id, modisdate, north, notify_change, observe, on_displayed, on_interaction, on_msg, on_trait_change, on_widget_constructed, open, options, remove_class, remove_control, remove_layer, scroll_wheel_zoom, send, send_state, set_state, set_trait, setup_instance, south, substitute_layer, tap, tap_tolerance, touch_zoom, trait_events, t

As we can see, this map object can do a lot. For example, you can set and get the center of the map.

In [59]:
# Get the current map center and zoom level.
print('center coords = {0}'.format(map1.center))
print('zoom = {0}'.format(map1.zoom))

center coords = [0.0, 0.0]
zoom = 2


In [60]:
# Set the current map center and zoom level (to the San Francisco Bay Area)
map1.center = [37.9, -122.0]
map1.zoom =6

Create a second map, using a custom basemap.

In [61]:
def getTileLayerUrl(map_id):
  template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return template.format(**map_id)

custom_basemap_mapid = ee.Image('NOAA/NGDC/ETOPO1').select('bedrock').gt(0).getMapId()

map2 = ipyleafletee.Map(
  zoom=2,
  center=[37.9, -122.0],
  default_tiles=ipyleafletee.TileLayerEE(
    url=getTileLayerUrl(custom_basemap_mapid)
  )
)
map2


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

Add an overlay layer to the map.

In [62]:
gsw = ee.Image('JRC/GSW1_0/GlobalSurfaceWater')
occurrence = gsw.select('occurrence')
map_id = occurrence.visualize(
  min=0,
  max=100,
  palette= ['red', 'blue']
).updateMask(occurrence.divide(100)).getMapId()

jrc_tile_url = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}".format(**map_id)
tl_jrc = ipyleafletee.TileLayer(url=jrc_tile_url)
map2.add_layer(tl_jrc)